In [ ]:
import numpy as np
import scipy
import datetime as dt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pickle
from multiprocessing import Pool
import os
import Rheology_fitting_toolkit as rft


/tmp/ipykernel_1226555/4207224591.py:16: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)



dictionary_all_events = {}
CREEPMETER = ['XHR']

for q in range(len(CREEPMETER)):
    print(CREEPMETER[q])
    tm, min10_creep, tm2, min10_creep2 = rft.import_text(CREEPMETER[q])

    if CREEPMETER[q] == 'XSJ' or CREEPMETER[q] == 'XHR' or CREEPMETER[q] == 'XPK':
        tm_int, min10_creep_int = rft.interpolate(tm,min10_creep,CREEPMETER)
        tm_int2, min10_creep_int2 = rft.interpolate(tm2,min10_creep2,CREEPMETER)
    elif CREEPMETER[q] == 'XMR':
        tm_int, min10_creep_int = rft.interpolate(tm,min10_creep,CREEPMETER)
        tm_int2, min10_creep_int2 = rft.interpolate_1min(tm2,min10_creep2,CREEPMETER)
    else:
        tm_int, min10_creep_int = rft.interpolate(tm,min10_creep,CREEPMETER)


    df_PICKS, duration, START = rft.creepmeter_events(CREEPMETER[q])

    if CREEPMETER[q] == 'XSJ' or CREEPMETER[q] == 'XHR' or CREEPMETER[q] == 'XPK':
        data1  = rft.vel_acc(tm_int,min10_creep_int,10/60)
        data2 = rft.vel_acc(tm_int2,min10_creep_int2,10/60)
        data = data1.append(data2,ignore_index=True)
    elif CREEPMETER[q] == 'XMR':
        data1  = rft.vel_acc(tm_int,min10_creep_int,10/60)
        data2 = rft.vel_acc_1min(tm_int2,min10_creep_int2,1/60)
        data = data1.append(data2,ignore_index=True)
    else:
        data = rft.vel_acc(tm_int,min10_creep_int,10/60)


    df_auto = rft.parkfield_remover(df_PICKS,CREEPMETER[q])


    df_rain_day_total = rft.rain_timeseries(CREEPMETER[q])

    df_auto = rft.when_does_it_rain(df_auto,CREEPMETER[q])
    
    if CREEPMETER[q] == 'CWN':
        C_matrix = np.load('../../Rheology/CWN/CWN_covariance_matrix_12days_18_APR_23.npy')
        C_matrix_inv_CWN = np.linalg.inv(C_matrix)
    
    if CREEPMETER[q] == 'XHR':
        C_matrix_2 = np.load('../../Rheology/XHR/XHR_2_covariance_matrix_4days_27_APR_23.npy')
        C_matrix_3 = np.load('../../Rheology/XHR/XHR_3_covariance_matrix_4days_27_APR_23.npy')
        C_matrix_inv_2 = np.linalg.inv(C_matrix_2)
        C_matrix_inv_3 = np.linalg.inv(C_matrix_3)
    
    dataframes_long, creep_index_long = rft.creep_event_dataframe(df_auto,duration, START, data,CREEPMETER[q])
    dataframes, creep_index = rft.creep_event_dataframe_short(dataframes_long,df_auto)

    Creep_phases = pd.read_csv("../../Rheology/{k}/Creep_phases_{k}.csv".format(k=CREEPMETER[q]),index_col=0)
    



XHR


In [ ]:
def VSF_aSS_fitter(time,slip,cov_matrix_inv,no_phases,columns_VSF_aSS,j,CREEPMETER,VSF_aSS_DF_params,atest,file_misfit):
    print('VSF_aSS: {k}'.format(k=j))
    print ("Current date and time : ")
    print (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    isExistVSF_aSS = os.path.exists('../../Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{y}/VSF_aSS/{k}_{y}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt'.format(k=CREEPMETER,y=j))
    if not isExistVSF_aSS:
        success = False
        n_iter = 5000
        VSF_aSS_DF_params['t1'].loc['initial'] = 2
        VSF_aSS_DF_params['t1'].loc['bounds'] = (-10,10)
        VSF_aSS_bounds = VSF_aSS_DF_params.loc['bounds'].to_list()
        VSF_aSS_initial_guess = VSF_aSS_DF_params.loc['initial'].to_list()
        
        while success == False:
            if success == False:
                res_VSF_aSS = scipy.optimize.basinhopping(rft.VSF_aSS_dromedary, VSF_aSS_initial_guess,\
                accept_test = atest, minimizer_kwargs = ({'args':(time,slip,cov_matrix_inv,no_phases,\
                columns_VSF_aSS,j,CREEPMETER,file_misfit),'method':'SLSQP','bounds':VSF_aSS_bounds}))#,'options':{'maxiter':n_iter}}),niter=1000)
                n_iter = n_iter+2000
                success = res_VSF_aSS.success
                VSF_aSS_initial_guess = res_VSF_aSS.x
        dictionary_VSF_aSS = {}
        dictionary_VSF_aSS['fit'] = res_VSF_aSS
        VSF_aSS_fitting_params = pd.DataFrame([res_VSF_aSS.x],columns = ('Ts','Vs','K','T01','S1','Ta1','V01','t1','T02','S2'), index = ['fitted'])
        VSF_aSS_DF_params = pd.concat([VSF_aSS_DF_params,VSF_aSS_fitting_params])
        dictionary_VSF_aSS['fitting params'] = VSF_aSS_DF_params
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/VSF_aSS/{k}_{u}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt".format(k=CREEPMETER,u=j), "wb") as tf:
            pickle.dump(dictionary_VSF_aSS,tf)

    else:
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/VSF_aSS/{k}_{u}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt".format(k=CREEPMETER,u=j), "rb") as tf:
            dictionary_VSF_aSS = pickle.load(tf)
            res_VSF_aSS = dictionary_VSF_aSS['fit']
            VSF_aSS_DF_params = dictionary_VSF_aSS['fitting params']
    return VSF_aSS_DF_params

In [ ]:
def test_multi(j):
    print(j)
    if len(dataframes_long[j].Time) <= 577:
        isExist = os.path.exists('/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/figures/SLSQP/{y}/{k}_{y}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.pdf'.format(k=CREEPMETER[q],y=j))
        if not isExist:
            isExist2 = os.path.exists('/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{y}/{k}_{y}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt'.format(k=CREEPMETER[q],y=j))
            if not isExist2:
                print('/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{y}/{k}_{y}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt'.format(k=CREEPMETER[q],y=j))
                Creep_Phase_no = Creep_phases.iloc[j].dropna()
                number_of_phases = (len(Creep_Phase_no)/2)-1
                if 1<= number_of_phases <=2:
                    data_P0, data_P1, data_P2, data_P3, data_P4, creep_phase_new = rft.phase_splitter(Creep_Phase_no,dataframes_long[j])

                    VSF_aSS_DF_params = rft.initial_and_bounds(creep_phase_new,data_P0,data_P1,data_P2,data_P3,data_P4,'VSF_aSS')
                    if CREEPMETER[q] == 'XHR':
                        if j < 70:
                            C_matrix_inv = C_matrix_inv_2
                        else:
                            C_matrix_inv = C_matrix_inv_3
                    else:
                        C_matrix_inv = C_matrix_inv_CWN

                    rft.check_dir("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}".format(k=CREEPMETER[q]))
                    Rheologies_to_test = ['LNV','PLV','VSF_SS','VSF_bSS','VSF_aSS','RDF']#,'cb77']
                    if number_of_phases==1:
                        VSF_aSS_DF_params.drop(['Ta2','V02','t2','T03','S3','Ta3','V03','t3','T04','S4','Ta4','V04','t4'], axis=1, inplace=True)

                        for z in range(len(Rheologies_to_test)):
                            rft.check_dir("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/{x}".format(k=CREEPMETER[q],u=j,x=Rheologies_to_test[z]))

                        VSF_aSS_params_tried = np.zeros(len(VSF_aSS_DF_params.loc['initial'].index.tolist()))
                        
                        columns_VSF_aSS = VSF_aSS_DF_params.loc['initial'].index.tolist()


                        rft.check_dir("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}".format(k=CREEPMETER[q],u=j))

                        def atest(f_new, x_new, f_old, x_old):
                            if f_old < f_new:
                                return False
                            else:
                                return True

                        
                        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/VSF_aSS/{k}_{u}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt".format(k=CREEPMETER[q],u=j),'a') as file_VSF_aSS:
                            VSF_aSS_DF_params = VSF_aSS_fitter(np.array(dataframes_long[j].Time),np.array(dataframes_long[j].Slip), C_matrix_inv,number_of_phases,columns_VSF_aSS,j,CREEPMETER[q],\
                                                               VSF_aSS_DF_params,atest,file_VSF_aSS)

    return

In [8]:
dictionary_all_events = {}
if __name__ == '__main__':
    with Pool(1) as pool:                         # Create a multiprocessing Pool           
        pool.map(test_multi, dataframes)

0
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/0/XHR_0_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
1
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/1/XHR_1_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
2
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/2/XHR_2_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 2
Current date and time : 
2023-08-24 16:14:27


/home/earthquakes1/software/Ubuntu/anaconda3/envs/dan2/lib/python3.10/site-packages/numba/core/ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'optimized_par' of function 'Velocity_strengthening_friction_3'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "Rheology_fitting_toolkit.py", line 75:
@jit(nopython=True,error_model = 'numpy')
def Velocity_strengthening_friction_3(optimized_par, OBS_Time):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/home/earthquakes1/software/Ubuntu/anaconda3/envs/dan2/lib/python3.10/site-packages/numba/core/ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'optimized_par' of function 'Velocity_strengthening_friction_3'.

For m

3
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/3/XHR_3_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
4
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/4/XHR_4_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 4
Current date and time : 
2023-08-24 16:14:30
5
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/5/XHR_5_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
6
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/6/XHR_6_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
7
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/7/XHR_7_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 7
Current date and time : 
2023-08-24 16:14:31
8
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/8/XHR_8_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 8
Current date and time : 
2023-0

/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/51/XHR_51_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
52
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/52/XHR_52_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 52
Current date and time : 
2023-08-24 16:15:05
53
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/53/XHR_53_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 53
Current date and time : 
2023-08-24 16:15:06
54
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/54/XHR_54_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
55
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/55/XHR_55_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
56
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/56/XHR_56_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 56
Current date 

/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/100/XHR_100_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
101
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/101/XHR_101_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 101
Current date and time : 
2023-08-24 16:15:24
102
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/102/XHR_102_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 102
Current date and time : 
2023-08-24 16:15:25
103
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/103/XHR_103_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
104
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/104/XHR_104_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
105
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/105/XHR_105_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aS

In [ ]:
def VSF_aSS_fitter(time,slip,cov_matrix_inv,no_phases,columns_VSF_aSS,j,CREEPMETER,VSF_aSS_DF_params,atest,file_misfit):
    print('VSF_aSS: {k}'.format(k=j))
    print ("Current date and time : ")
    print (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    isExistVSF_aSS = os.path.exists('../../Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{y}/VSF_aSS/{k}_{y}_VSF_aSS_t1_neg_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt'.format(k=CREEPMETER,y=j))
    if not isExistVSF_aSS:
        success = False
        n_iter = 5000
        VSF_aSS_DF_params['t1'].loc['bounds'] = (-10,10)
        VSF_aSS_bounds = VSF_aSS_DF_params.loc['bounds'].to_list()
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/VSF_aSS/{k}_{u}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt".format(k=CREEPMETER,u=j),'rb') as fname:
            VSF_aSS_SLSQP = pickle.load(fname)        
        VSF_aSS_initial_guess = VSF_aSS_SLSQP['fitting params'].loc['fitted'].to_list() 
        while success == False:
            if success == False:
                res_VSF_aSS = scipy.optimize.basinhopping(rft.VSF_aSS_dromedary, VSF_aSS_initial_guess,\
                accept_test = atest, minimizer_kwargs = ({'args':(time,slip,cov_matrix_inv,no_phases,\
                columns_VSF_aSS,j,CREEPMETER,file_misfit),'method':'Nelder-Mead','bounds':VSF_aSS_bounds}))#,'options':{'maxiter':n_iter}}),niter=1000)
                n_iter = n_iter+2000
                success = res_VSF_aSS.success
                VSF_aSS_initial_guess = res_VSF_aSS.x
        dictionary_VSF_aSS = {}
        dictionary_VSF_aSS['fit'] = res_VSF_aSS
        VSF_aSS_fitting_params = pd.DataFrame([res_VSF_aSS.x],columns = ('Ts','Vs','K','T01','S1','Ta1','V01','t1','T02','S2'), index = ['fitted'])
        VSF_aSS_DF_params = pd.concat([VSF_aSS_DF_params,VSF_aSS_fitting_params])
        dictionary_VSF_aSS['fitting params'] = VSF_aSS_DF_params
        #dictionary_VSF_aSS['params tried'] = VSF_aSS_params_tried
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/VSF_aSS/{k}_{u}_VSF_aSS_t1_neg_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "wb") as tf:
            pickle.dump(dictionary_VSF_aSS,tf)

    else:
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/VSF_aSS/{k}_{u}_VSF_aSS_t1_neg_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "rb") as tf:
            dictionary_VSF_aSS = pickle.load(tf)
            res_VSF_aSS = dictionary_VSF_aSS['fit']
            VSF_aSS_DF_params = dictionary_VSF_aSS['fitting params']
    return VSF_aSS_DF_params

In [10]:
dictionary_all_events = {}
if __name__ == '__main__':
    with Pool(1) as pool:                         # Create a multiprocessing Pool           
        pool.map(test_multi, dataframes)

0
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/0/XHR_0_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
1
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/1/XHR_1_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
2
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/2/XHR_2_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 2
Current date and time : 
2023-08-24 16:16:06


/home/earthquakes1/software/Ubuntu/anaconda3/envs/dan2/lib/python3.10/site-packages/numba/core/ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'optimized_par' of function 'Velocity_strengthening_friction_3'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "Rheology_fitting_toolkit.py", line 75:
@jit(nopython=True,error_model = 'numpy')
def Velocity_strengthening_friction_3(optimized_par, OBS_Time):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/home/earthquakes1/software/Ubuntu/anaconda3/envs/dan2/lib/python3.10/site-packages/numba/core/ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'optimized_par' of function 'Velocity_strengthening_friction_3'.

For m

3
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/3/XHR_3_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
4
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/4/XHR_4_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 4
Current date and time : 
2023-08-24 16:16:42
5
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/5/XHR_5_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
6
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/6/XHR_6_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
7
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/7/XHR_7_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 7
Current date and time : 
2023-08-24 16:17:39
8
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/8/XHR_8_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 8
Current date and time : 
2023-0

/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/51/XHR_51_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
52
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/52/XHR_52_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 52
Current date and time : 
2023-08-24 16:48:16
53
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/53/XHR_53_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 53
Current date and time : 
2023-08-24 16:49:40
54
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/54/XHR_54_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
55
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/55/XHR_55_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
56
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/56/XHR_56_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 56
Current date 

/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/100/XHR_100_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
101
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/101/XHR_101_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 101
Current date and time : 
2023-08-24 17:13:18
102
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/102/XHR_102_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aSS: 102
Current date and time : 
2023-08-24 17:14:10
103
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/103/XHR_103_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
104
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/104/XHR_104_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
105
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/SLSQP/105/XHR_105_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt
VSF_aS

In [11]:
with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/VSF_aSS/{k}_{u}_VSF_aSS_t1_neg_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER[0],u=105),'rb') as fname:
    VSF_aSS_SLSQP = pickle.load(fname) 

In [12]:
VSF_aSS_SLSQP

{'fit':                         fun: 0.021925160164225847
  lowest_optimization_result:  final_simplex: (array([[ 2.48807501e+00,  2.08973061e-01, -3.53373676e-01,
          3.33333333e+00,  3.88724023e-01,  9.99993428e+01,
          1.62299994e+00,  8.05080886e-02,  2.11666667e+01,
          1.40100000e+00],
        [ 2.48805516e+00,  2.08971771e-01, -3.53375545e-01,
          3.33333333e+00,  3.88724023e-01,  9.99993937e+01,
          1.62299995e+00,  8.05088674e-02,  2.11666667e+01,
          1.40100000e+00],
        [ 2.48806145e+00,  2.08972101e-01, -3.53374380e-01,
          3.33333333e+00,  3.88724023e-01,  9.99994081e+01,
          1.62299995e+00,  8.05075506e-02,  2.11666667e+01,
          1.40100000e+00],
        [ 2.48804839e+00,  2.08971324e-01, -3.53375523e-01,
          3.33333333e+00,  3.88724023e-01,  9.99994189e+01,
          1.62299994e+00,  8.05079716e-02,  2.11666667e+01,
          1.40100000e+00],
        [ 2.48810702e+00,  2.08975166e-01, -3.53371052e-01,
        

In [ ]:
any_t1 = []
t1_2 = []
for i in range(len(dataframes_long)):
    try:
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/Nelder-Mead/{i}/VSF_aSS/XHR_{i}_VSF_aSS_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(i=i), "rb") as tf:
            dictionary_VSF_aSS = pickle.load(tf)
            any_t1.append(dictionary_VSF_aSS['fit'].fun)
    except FileNotFoundError:
        continue
    try:
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/Nelder-Mead/{i}/VSF_aSS/XHR_{i}_VSF_aSS_t1_neg_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(i=i), "rb") as tf:
            dictionary_VSF_aSS = pickle.load(tf)
            t1_2.append(dictionary_VSF_aSS['fit'].fun)
            print(dictionary_VSF_aSS['fitting params']['t1'].loc['fitted'])
    except FileNotFoundError:
        continue

0.2643491911091042
0.18902968835113804
0.11975966104007213
0.26921133561340727
0.14418003648832217
0.23295742157707366
0.19576093861575655
0.1088694587601333
0.18154727432530068
0.18496100333983298
0.11748957980386693
0.08471959127586115
0.08912563259560691
0.12491534622364264
0.11676062710230464
0.07493317844704975
0.11592062405574935
0.09394277477293855
0.09712244548516133
0.07896453377995011
0.08098648759739766
0.26430851640061515
0.5529198440546674
0.06540521807426353
0.2577619399753093
0.09576522194084627
0.1402754888631087
0.10118717023534954
0.1777373871803186
0.1336303079355782
0.10926174195716543
0.0628655927537988
0.13805094017048933
0.03610124015974987
0.09371721689668995
-10.0
0.11519692130038939
0.06461736772310489
0.2071392959400625
0.9801103794279077
0.09799747623804034
0.4352548315325848
0.13023608824538263
0.07330643730995881
0.21812718277242837
0.14653305577296943
0.07279252354350485
0.21926464335178938
0.20824978285051285
0.08050808861770709
